### Toy Random Walker

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def update_polarity(v, mag, D, dt):
    theta = np.arctan2(v[1], v[0])
    theta += np.random.normal(0, 1) * np.sqrt(2 * D * dt)
    return mag * np.array([np.cos(theta), np.sin(theta)])


def init_cell_polarity():
    theta = np.random.uniform(0, 2 * np.pi)
    return 10 * np.array([np.cos(theta), np.sin(theta)])


def MSD(X, upper_lim=1000):
    msd = []
    lag_times = np.arange(1, X.shape[0] - upper_lim)
    for n in lag_times:
        dr = X[n:] - X[:-n]
        msd.append((np.linalg.norm(dr, axis=1) ** 2).mean())
    return lag_times, np.array(msd)

In [ ]:
D = 5
dt = 0.01
mag = 10
N = 10000

r = [[0, 0]]
v = [init_cell_polarity()]

for n in range(N):
    v_new = update_polarity(v[n], mag, D, dt)
    v.append(v_new)
    r.append(r[n] + v[n] * dt)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(5, 1.5), dpi=200)

r = np.array(r)
axs[0].scatter(r[:, 0], r[:, 1], c=range(r.shape[0]), s=5)

lag_times, msd = MSD(r)
lag_times = lag_times * dt
a = 5000
axs[1].plot(lag_times, msd, color="black", lw=2)
axs[1].plot(
    lag_times[100:],
    a * lag_times[100:],
    lw=1,
    ls="dashed",
    color="salmon",
    label=r"$\sim \tau$",
)
axs[1].plot(
    lag_times[:50],
    a * lag_times[:50] ** 2,
    lw=1,
    ls="dashed",
    color="cornflowerblue",
    label=r"$\sim \tau^2$",
)
axs[1].set_xscale("log")
axs[1].set_yscale("log")
axs[1].set_xscale("log")
axs[1].set_xlabel("Lag time " + r"$\tau$")
axs[1].set_ylabel("MSD")
axs[1].legend(bbox_to_anchor=(1, 1))

fig.subplots_adjust(wspace=0.75)
plt.show()

---

### CompuCell Random Walker

In [ ]:
from glob import glob

fig, ax = plt.subplots(1, 1, figsize=(2, 2), dpi=200)
cmap = plt.get_cmap("tab10")

for k, file in enumerate(glob("output/*")):
    cm_track = np.load(open(file, "rb"))
    ax.scatter(*cm_track[0], s=100, marker="*", color=cmap(k))
    ax.scatter(*cm_track[-1], s=100, marker="+", color=cmap(k))
    ax.scatter(cm_track[:, 0], cm_track[:, 1], s=0.1, color=cmap(k), alpha=0.05)

ax.set_xlabel(r"$x$")
ax.set_ylabel(r"$y$")
# ax.set_xlim((0, 100))
# ax.set_ylim((0, 100))
plt.show()

In [ ]:
dt = 0.01
cm_track = np.load(open("output/com_track_cellID_1.npy", "rb"))
print(cm_track.shape)

lag_times, msd = MSD(cm_track)
lag_times = lag_times * dt
x = np.arange(msd.size)

fig, axs = plt.subplots(1, 2, figsize=(5, 1.5), dpi=200)
axs[0].scatter(
    cm_track[:, 0],
    cm_track[:, 1],
    c=np.arange(cm_track.shape[0]),
    cmap="viridis",
    s=5,
)
axs[0].set_xlabel(r"$x$")
axs[0].set_ylabel(r"$y$")
# axs[0].set_xlim((0, 500))
# axs[0].set_ylim((0, 500))

a = 5000
axs[1].plot(lag_times, msd, color="black", lw=2)
axs[1].plot(
    lag_times[100:],
    a * lag_times[100:],
    lw=1,
    ls="dashed",
    color="salmon",
    label=r"$\sim \tau$",
)
axs[1].plot(
    lag_times[:50],
    a * lag_times[:50] ** 2,
    lw=1,
    ls="dashed",
    color="cornflowerblue",
    label=r"$\sim \tau^2$",
)
axs[1].set_xscale("log")
axs[1].set_yscale("log")
axs[1].set_xscale("log")
axs[1].set_xlabel("Lag time " + r"$\tau$")
axs[1].set_ylabel("MSD")
axs[1].vlines(
    1 / (1.5),
    *plt.ylim(),
    lw=1,
    color="black",
    ls="dashed",
    label=r"$1/D_{\theta}$",
)
axs[1].legend(bbox_to_anchor=(1, 1))

fig.subplots_adjust(wspace=0.75)
plt.show()